In [1]:
import numpy as np
from __future__ import annotations


from typing import Optional, Sequence, Tuple
import math
import keras_core as keras
from keras_core import ops
import tensorflow as tf

import sys
sys.path.append('./../../scripts/nn_train_size_analysis/')
from rtdl_num_embeddings_keras import (
    PeriodicEmbeddings,
    PiecewiseLinearEmbeddings,
    PiecewiseLinearEncoding,
)

Using TensorFlow backend


2025-10-14 20:53:27.639325: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-14 20:53:27.639380: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-14 20:53:27.640727: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-14 20:53:27.651168: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-14 20:53:30.322774: W tensorflow/comp

In [2]:
# Deterministic RNG across backends
try:
    keras.utils.set_random_seed(42)
except Exception:
    pass
seed = 5
np.random.seed(seed)
n_features = 5
batch = 1

x = 5 * np.random.randn(batch, n_features).astype("float32")
print(x)

[[ 2.2061374 -1.6543508 12.153855  -1.2604606  0.5480492]]


In [6]:
# Test PiecewiseLinearEncoding on x with a set value_range
range = (-2.0, 2.0)
mins = np.ones(n_features, dtype="float32") * range[0]
maxs = np.ones(n_features, dtype="float32") * range[1]
value_range = (mins, maxs)
n_features = x.shape[1]
n_bins = 4

x_tensor = ops.convert_to_tensor(x)

import time
ple = PiecewiseLinearEncoding(
    value_range=value_range,
    n_features=n_features,
    n_bins=n_bins,
    use_adaptive_range=False,
    clip=False,
)

start = time.time()

y = ple(x_tensor)

end = time.time()
print(f"Time taken: {end - start} seconds")
print(y)

# Undo the concatenation of shape (batch, n_features * n_bins)
y = tf.reshape(y, (batch, n_features, n_bins))
print(y)

Time taken: 0.02169013023376465 seconds
tf.Tensor(
[[ 1.          1.          1.          1.2061377   0.34564924  0.
   0.          0.          1.          1.          1.         11.153855
   0.7395394   0.          0.          0.          1.          1.
   0.5480492   0.        ]], shape=(1, 20), dtype=float32)
tf.Tensor(
[[[ 1.          1.          1.          1.2061377 ]
  [ 0.34564924  0.          0.          0.        ]
  [ 1.          1.          1.         11.153855  ]
  [ 0.7395394   0.          0.          0.        ]
  [ 1.          1.          0.5480492   0.        ]]], shape=(1, 5, 4), dtype=float32)


In [13]:
# ===== PeriodicEmbeddings: shape + forward =====
per = PeriodicEmbeddings(
    n_features=n_features,
    n_frequencies=8,
    learnable_frequencies=True,
    use_phase=True,
    learnable_phases=True,
)
y_per = per(x)
expected_per = n_features * 8 * 2
assert y_per.shape == (batch, expected_per), f"Periodic shape {y_per.shape} != (B,{expected_per})"
print("PeriodicEmbeddings forward OK:", y_per.shape)

# Trainability check by fitting a tiny head
inp = keras.Input((n_features,), dtype="float32")
out = per(inp)
out = keras.layers.Dense(4, activation="relu")(out)
model_per = keras.Model(inp, out)
model_per.compile(optimizer="adam", loss="mse")
y_target = np.random.randn(batch, 4).astype("float32")
hist = model_per.fit(x, y_target, epochs=2, verbose=0)
assert np.isfinite(hist.history["loss"][ -1 ]), "NaN loss in PeriodicEmbeddings model"
print("PeriodicEmbeddings training step OK, loss:", hist.history["loss"][ -1 ])

# Serialization round-trip
model_per.save("per_test.keras")
model_per_loaded = keras.models.load_model(
    "per_test.keras", custom_objects={"PeriodicEmbeddings": PeriodicEmbeddings}
)
p1 = model_per.predict(x, verbose=0)
p2 = model_per_loaded.predict(x, verbose=0)
assert np.allclose(p1, p2, atol=1e-5), "Loaded Periodic model predictions differ"
print("PeriodicEmbeddings serialization OK")



PeriodicEmbeddings forward OK: (1, 80)


2025-10-14 20:29:29.757231: I external/local_xla/xla/service/service.cc:168] XLA service 0x15371c054670 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-10-14 20:29:29.757256: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2025-10-14 20:29:29.823051: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-10-14 20:29:30.355620: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1760509770.746177 1203056 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


PeriodicEmbeddings training step OK, loss: 0.8265498280525208
PeriodicEmbeddings serialization OK


In [14]:
# ===== PiecewiseLinearEncoding (fixed): adapt + forward =====
ple = PiecewiseLinearEncoding(
    n_features=n_features,
    n_bins=16,
    use_adaptive_range=True,
    clip=True,
)
ple.adapt(x)
y_ple = ple(x)
expected_ple = n_features * 16
assert y_ple.shape == (batch, expected_ple), f"PLE shape {y_ple.shape} != (B,{expected_ple})"
print("PiecewiseLinearEncoding forward OK:", y_ple.shape)

# PLE can feed an MLP head
inp2 = keras.Input((n_features,), dtype="float32")
out2 = ple(inp2)
out2 = keras.layers.Dense(3)(out2)
model_ple = keras.Model(inp2, out2)
model_ple.compile(optimizer="adam", loss="mse")
y2 = np.random.randn(batch, 3).astype("float32")
hist2 = model_ple.fit(x, y2, epochs=2, verbose=0)
assert np.isfinite(hist2.history["loss"][ -1 ]), "NaN loss in PLE model"
print("PiecewiseLinearEncoding training step OK, loss:", hist2.history["loss"][ -1 ])

# Serialization round-trip (PLE)
model_ple.save("ple_fixed_test.keras")
model_ple_loaded = keras.models.load_model(
    "ple_fixed_test.keras",
    custom_objects={"PiecewiseLinearEncoding": PiecewiseLinearEncoding}
)
q1 = model_ple.predict(x, verbose=0)
q2 = model_ple_loaded.predict(x, verbose=0)
assert np.allclose(q1, q2, atol=1e-5), "Loaded PLE model predictions differ"
print("PiecewiseLinearEncoding serialization OK")

PiecewiseLinearEncoding forward OK: (1, 80)
PiecewiseLinearEncoding training step OK, loss: 0.5233413577079773
PiecewiseLinearEncoding serialization OK


In [15]:
# ===== PiecewiseLinearEmbeddings: adapt + forward =====
plemb = PiecewiseLinearEmbeddings(
    n_features=n_features,
    n_bins=16,
    d_embedding=8,
    activation=True,          # ReLU(Linear(PLE))
    use_adaptive_range=True,
    clip=True,
)
plemb.adapt(x)
y_plemb = plemb(x)
expected_plemb = n_features * 8
assert y_plemb.shape == (batch, expected_plemb), f"PLEmb shape {y_plemb.shape} != (B,{expected_plemb})"
print("PiecewiseLinearEmbeddings forward OK:", y_plemb.shape)

# Trainability + compile/fit for embeddings variant
inp3 = keras.Input((n_features,), dtype="float32")
out3 = plemb(inp3)
out3 = keras.layers.Dense(3)(out3)
model_plemb = keras.Model(inp3, out3)
model_plemb.compile(optimizer="adam", loss="mse")
y3 = np.random.randn(batch, 3).astype("float32")
hist3 = model_plemb.fit(x, y3, epochs=2, verbose=0)
assert np.isfinite(hist3.history["loss"][ -1 ]), "NaN loss in PLEmb model"
print("PiecewiseLinearEmbeddings training step OK, loss:", hist3.history["loss"][ -1 ])

# Serialization round-trip (PLEmb)
model_plemb.save("plemb_test.keras")
model_plemb_loaded = keras.models.load_model(
    "plemb_test.keras",
    custom_objects={
        "PiecewiseLinearEmbeddings": PiecewiseLinearEmbeddings,
        "PiecewiseLinearEncoding": PiecewiseLinearEncoding,
    }
)
r1 = model_plemb.predict(x, verbose=0)
r2 = model_plemb_loaded.predict(x, verbose=0)
assert np.allclose(r1, r2, atol=1e-5), "Loaded PLEmb model predictions differ"
print("PiecewiseLinearEmbeddings serialization OK")

PiecewiseLinearEmbeddings forward OK: (1, 40)
PiecewiseLinearEmbeddings training step OK, loss: 1.0864019393920898
PiecewiseLinearEmbeddings serialization OK


In [16]:
print("All quick tests passed ✔")

All quick tests passed ✔


# Try building a model and passing data through

In [5]:
import os
import datetime
import h5py
import numpy as np
import optuna
import tensorflow as tf
import tensorflow_io as tfio
import keras_core as keras
from tensorflow.data import Dataset
from tensorflow.data.experimental import AUTOTUNE
from optuna.storages import JournalStorage, JournalFileStorage

import sys
sys.path.append('./../../scripts/nn_train_size_analysis/')
from rtdl_num_embeddings_keras import (
    PeriodicEmbeddings,
    PiecewiseLinearEncoding,
    PiecewiseLinearEmbeddings,
)

/home/linneamw/anaconda3/envs/gcr-gpu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_dataset(polarity, data_version, train_size_fraction, bootstrap):
    # 8 input parameters for the NN: alpha, cmf, vspoles, cpa, pwr1par, pwr2par, pwr1perr, and pwr2perr.
    # features = ['alpha', 'cmf', 'cpa', 'pwr1par', 'pwr1perr', 'pwr2par', 'pwr2perr', 'vspoles']
    data_path = '/home/linneamw/sadow_koastore/personal/linneamw/research/gcr/data/shuffled_may2025'
    train_file = f'{data_path}/{polarity}/train.h5'
    test_file = f'{data_path}/{polarity}/test.h5'

    # Load train data
    with h5py.File(train_file, 'r') as h5:
        num_train_samples, num_inputs,  = h5['X_minmax'].shape
        _, num_flux,  = h5['Y_log_scaled'].shape
    x_train = tfio.IODataset.from_hdf5(train_file, dataset='/X_minmax')
    y_train = tfio.IODataset.from_hdf5(train_file, dataset='/Y_log_scaled')
    full_train = Dataset.zip((x_train, y_train))

    # Load test data
    with h5py.File(test_file, 'r') as h5:
        num_test_samples, num_inputs,  = h5['X_minmax'].shape
        _, num_flux,  = h5['Y_log_scaled'].shape
    x_test = tfio.IODataset.from_hdf5(test_file, dataset='/X_minmax')
    y_test = tfio.IODataset.from_hdf5(test_file, dataset='/Y_log_scaled')
    test = Dataset.zip((x_test, y_test))

    # Get number of training samples (from the dataset)
    train_size = int(np.floor(num_train_samples * train_size_fraction))
    print(f"Number of training samples: {train_size} out of {num_train_samples} total")
    print(f"Number of test samples: {num_test_samples}")

    # Choose seed based on model version
    data_seeds = {
        'd1': 42,
        'd2': 87,
        'd3': 5,
        'd4': 98,
        'd5': 123,
    }
    data_seed = data_seeds.get(data_version, None)

    if bootstrap == 'b1':
        # Reproducible bootstrap indices
        rng = np.random.default_rng(data_seed)
        sampled_indices = rng.integers(low=0, high=num_train_samples, size=train_size)

        # Load dataset into memory
        train_list = list(full_train.as_numpy_iterator())

        # Sample with replacement
        bootstrapped_data = [train_list[i] for i in sampled_indices]

        # Separate into inputs and outputs
        x_bootstrap, y_bootstrap = zip(*bootstrapped_data)

        # Convert back to tf.data.Dataset
        train = Dataset.from_tensor_slices((list(x_bootstrap), list(y_bootstrap)))

    else:
        # Shuffle deterministically
        if data_version in data_seeds:
            train_shuffled = full_train.shuffle(
                buffer_size=num_train_samples, seed=data_seed, reshuffle_each_iteration=False
            )
        else:
            train_shuffled = full_train

        # Take subset without replacement
        train = train_shuffled.take(train_size)

    # Set batch_size to 128 unless the train size is smaller than 128, then set it to the train size.
    if train_size < 128:
        batch_size = train_size
    else:
        batch_size = 128

    train = train.batch(batch_size, drop_remainder=True).prefetch(AUTOTUNE)
    test = test.batch(batch_size, drop_remainder=True).prefetch(AUTOTUNE)

    return train, test, train_size, num_test_samples, batch_size, num_inputs

def build_model(input_dim, n_layers, units, embedding_method, embed_dim=12, n_bins=48, n_frequencies=8, value_range=None):
    print(f"Building model with embedding {embedding_method}, {n_layers} layers, {units} units per layer, embed_dim {embed_dim}, n_bins {n_bins}, n_frequencies {n_frequencies}, value_range {value_range}")

    model = keras.Sequential([keras.Input(shape=(input_dim,), dtype="float32")])

    # Tabular embedding layer
    if embedding_method == "periodic":
        # Defaults: k=64, sigma=0.02, activation=True (you can change)
        model.add(PeriodicEmbeddings(
            n_features=input_dim,
            n_frequencies=n_frequencies,
            learnable_frequencies=True,
            use_phase=True,
            learnable_phases=True,
        ))
        model.add(keras.layers.Flatten())
    elif embedding_method == "piecewise_encoding":
        if value_range is None:
            model.add(PiecewiseLinearEncoding(
                n_features=input_dim,
                n_bins=n_bins,
                use_adaptive_range=True,
                clip=True,
            ))
        else:
            model.add(PiecewiseLinearEncoding(
                n_features=input_dim,
                n_bins=n_bins,
                use_adaptive_range=False,
                value_range=value_range,
                clip=True,
            ))
    elif embedding_method == "piecewise_embedding":
        if value_range is None:
            model.add(PiecewiseLinearEmbeddings(
                n_features=input_dim,
                n_bins=n_bins,
                d_embedding=embed_dim,
                activation=True,          # ReLU(Linear(PLE))
                use_adaptive_range=True,
                clip=True,
            ))
        else:
            model.add(PiecewiseLinearEmbeddings(
                n_features=input_dim,
                n_bins=n_bins,
                d_embedding=embed_dim,
                activation=True,          # ReLU(Linear(PLE))
                use_adaptive_range=False,
                value_range=value_range,
                clip=True,
            ))
        model.add(keras.layers.Flatten())
    else:
        # No embedding, use raw inputs
        pass

    # If you’re using SELU, pair with lecun_normal + AlphaDropout (recommended for SELU)
    for _ in range(n_layers):
        model.add(keras.layers.Dense(units, activation="selu", kernel_initializer="lecun_normal"))
        # optional:
        # model.add(keras.layers.AlphaDropout(0.05))

    model.add(keras.layers.Dense(32, activation="linear"))
    return model

In [3]:
# Fixed args – customize if needed
args = {
    "polarity": "neg",
    "train_size_fraction": 0.01,
    "bootstrap": "b0",
    "data_version": "d1"
}

# Sample hyperparameters
learning_rate = 1e-3
weight_decay = 1e-5

train, test, train_size, num_test_samples, batch_size, num_inputs = load_dataset(
    args["polarity"], args["data_version"], args["train_size_fraction"], args["bootstrap"]
)

# For piecewise linear embeddings, we need to provide the value range (min, max) for each feature.
# All input data is min-max scaled already
n_features = 8
mins = np.zeros(n_features, dtype="float32")
maxs = np.ones(n_features, dtype="float32")
value_range = (mins, maxs)

# Get trial hyperparameters
n_layers = 3
units = 2048
embedding_method = "piecewise_embedding"
n_frequencies = 8
n_bins = 8
embed_dim = 16

# Define model
model = build_model(
    num_inputs, 
    n_layers, 
    units, 
    embedding_method,
    embed_dim=embed_dim, 
    n_frequencies=n_frequencies, 
    n_bins=n_bins, 
    value_range=value_range
)
print(model.summary())

# Compile model
optimizer = keras.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay)
model.compile(optimizer=optimizer, loss='mae', metrics=['mse'])

# Train
history = model.fit(
    train,
    epochs=2,
    validation_data=test,
    shuffle=False,
    verbose=2,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10),
    ]
)

# Evaluate on train
train_results = model.evaluate(train, verbose=0)
train_mae = train_results[0]  # This is the loss (MAE)
train_mse = train_results[1]  # This is the metric (MSE)

# Evaluate on test
test_results = model.evaluate(test, verbose=0)
test_mae = test_results[0]
test_mse = test_results[1]

print(f"Train MAE: {train_mae}, Train MSE: {train_mse}")
print(f"Test MAE: {test_mae}, Test MSE: {test_mse}")

2025-10-07 16:35:19.105579: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2025-10-07 16:35:19.110596: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available
2025-10-07 16:35:19.699336: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9614 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1b:00.0, compute capability: 7.5


Number of training samples: 17888 out of 1788892 total
Number of test samples: 198766
Building model with embedding piecewise_embedding, 3 layers, 2048 units per layer, embed_dim 16, n_bins 8, n_frequencies 8, value_range (array([0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ piecewise_linear_embeddings     │ (None, 128)               │      1,169 │
│ (PiecewiseLinearEmbeddings)     │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten (Flatten)               │ (None, 128)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense (Dense)                   │ (None, 2048)              │    264,192 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_1 (Dense)                 │ (None, 2048)              │  4,196,352 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_2 (Dense)                 │ (None, 2048)              │  4,196,352 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_3 (Dense)                 │ (None, 32)                │     65,568 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 8,723,633 (33.28 MB)

...

139/139 - 514s - 4s/step - loss: 0.2186 - mse: 0.4780 - val_loss: 0.0389 - val_mse: 0.0024 - learning_rate: 0.0010
Epoch 2/2


2025-10-07 16:45:04.674622: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:34: Filling up shuffle buffer (this may take a while): 110593 of 1788892
2025-10-07 16:45:15.917097: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:34: Filling up shuffle buffer (this may take a while): 114689 of 1788892
2025-10-07 16:45:32.288268: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:34: Filling up shuffle buffer (this may take a while): 117761 of 1788892
2025-10-07 16:45:57.030696: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:34: Filling up shuffle buffer (this may take a while): 132097 of 1788892
2025-10-07 16:46:10.757010: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:34: Filling up shuffle buffer (this may take a while): 134145 of 1788892
2025-10-07 16:46:21.593224: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:34: Filling up sh

139/139 - 264s - 2s/step - loss: 0.0316 - mse: 0.0016 - val_loss: 0.0247 - val_mse: 9.6294e-04 - learning_rate: 0.0010


2025-10-07 16:48:24.870583: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6033276781302955739
2025-10-07 16:48:24.870618: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8127168858024198982
2025-10-07 16:49:17.492152: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6033276781302955739
2025-10-07 16:49:17.492203: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8127168858024198982
2025-10-07 16:49:20.464015: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 6033276781302955739
2025-10-07 16:49:20.464051: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8127168858024198982


Train MAE: 0.024613454937934875, Train MSE: 0.000958481163252145
Test MAE: 0.024706387892365456, Test MSE: 0.000962943013291806
